# Imports

Besides external libraries, we import several function from the local file : `utils.py`

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import brightway2 as bw

# Custom utils defined for inter-acv
from utils import *
from expression import *

# Init brightway2 and databases

In [2]:
# Setup bw2
bw.projects.set_current('B_Publication')
bw.bw2setup()

# Import Ecoinvent DB (if not already done)
# Update the PATH to suit your installation
importDb(ECOINVENT_DB_NAME, './ecoinvent 3.4_cutoff_ecoSpold02/datasets')

# We use a separate DB for defining our model, reset it beforehand
resetDb(ACV_DB_NAME)

# Parameters are stored at project level : reset them
resetParams()

Biosphere database already present!!! No setup is needed


Database 'ecoinvent 3.4 cut off' has already been imported 
Db incer-acv was here. Reseting it


# Define parameters

We define the parameters of the model.

In [3]:
# Example 'float' parameter
alpha = newParamDef(
    'alpha', 
    ParamType.NUMBER, 
    default=0.5, 
    description="hello world")

beta = newParamDef(
    'beta', 
    ParamType.NUMBER, 
    default=0.5, 
    description="foo bar")

# Example 'enum' parameter, acting like a switch between several possibilities
elect_switch_param = newParamDef(
    'elect_switch_param', 
    ParamType.ENUM, 
    values=["us", "glo"], 
    default="us", 
    description="Switch on electricty mix")

# Get references to background activities and product

`utils` provide two functions for easy and fast search of activities in reference databases : 
* **findBioAct** : Search activity in **biosphere3** db
* **findTechAct** : Search activity in **ecoinvent** db


In [6]:
# Biosphere activities
occup = findBioAct('Occupation, industrial area')
heat = findBioAct('Heat, waste', categories=['air'])

# Technosphere activities
alu = findTechAct("aluminium alloy production", "RER")

# Define the model

In [19]:
# Create new, parametrized activity
activity1 = newActivity(
    name="act1", unit="kg", # Any extra named arg will be added as activity attribute
    exchanges= { # We define exhanges as a dict of 'activity : amount'
        occup:3 + alpha, # Amount can be a fixed value 
        heat: alpha}) # Amount can be a Sympy expression (any arithmetic expression of Parameters)

activity2 = newActivity(
    name="act2", unit="kg", 
    exchanges= {
        activity1: beta * 5 + alpha, # Reference the activity we just created
        heat: 7 - alpha,
        alu:0.4 / alpha }) 

Activity 'act1' was already in 'incer-acv'. Overwriting it
Activity 'act2' was already in 'incer-acv'. Overwriting it


In [8]:
print_act(activity1) 
print_act(activity2)
          
# Note that symbolic expressions have not been evaluated at this stage

,amount /formula,unit,type
"'Occupation, industrial area' (square meter-year, None, ('natural resource', 'land'))",alpha + 3,square meter-year,biosphere
"'Heat, waste' (megajoule, None, ('air',))",alpha,megajoule,biosphere


,amount /formula,unit,type
"'act1' (kg, GLO, None)",alpha + 5*beta,kg,technosphere
"'Heat, waste' (megajoule, None, ('air',))",7 - alpha,megajoule,biosphere
"'aluminium alloy production, AlMg3' (kilogram, RER, None)",0.4/alpha,kilogram,technosphere


In [9]:
elec_switch_activity = switch(
    elect_switch_param, dict(
        us=us_elec, 
        glo=glo_elec))

print_act(elec_switch_activity)

NameError: name 'us_elec' is not defined

# Select the impacts to consider

In [10]:
# List of impacts to consider
impacts = [m for m in bw.methods if 'ILCD 1.0.8 2016' in str(m) and 'no LT' in str(m)]

# Compute LCA

We provide two methods to compute LCA : 
* **multiLCAWithParams** : it uses brightway2 parametric capabilities
* **multiLCAWithParamsAlgebric** : This implementation allows list of sample values as parameter values. 
It computes an algebric expression of the model and compute LCA once for the background activities. Then it evaluates each impact with numpy compilation of the expression, provided by Sympy : It enables to compute whole list of sample values at once. It is extremely fast (more than 1 million time faster than using brightway2 parameters) : 

In [11]:
# Uses brightway2 parameters
multiLCAWithParams(
    activity2, # The model 
    impacts, # Impacts
    
    # Parameters of the model
    alpha=1, 
    beta=2)

,act2
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), resources, mineral, fossils and renewables)",4.025258e-03
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), climate change, GWP 100a)",2.292553e+00
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, freshwater and terrestrial acidification)",1.423843e-02
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, freshwater ecotoxicity)",2.356163e+00
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, freshwater eutrophication)",1.408405e-04
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, ionising radiation)",4.815676e-07
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, marine eutrophication)",1.910615e-03
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), ecosystem quality, terrestrial eutrophication)",2.173546e-02
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), human health, carcinogenic effects)",1.184520e-07
"(ILCD 1.0.8 2016 midpoint no LT (obsolete), human health, ionising radiation)",5.761168e-02


In [22]:
# Compute with algebric implementation : the values should be the same
multiLCAWithParamsAlgebric(
    activity2, # The model 
    impacts, # Impacts
    
    # Parameters of the model
    alpha=1, 
    beta=2)

Activity 'Occupation, industrial areacopy' was already in 'incer-acv'. Overwriting it
Activity 'Heat, wastecopy' was already in 'incer-acv'. Overwriting it


,"mineral, fossils and renewables",GWP 100a,freshwater and terrestrial acidification,freshwater ecotoxicity,freshwater eutrophication,ionising radiation,marine eutrophication,terrestrial eutrophication,carcinogenic effects,ionising radiation,non-carcinogenic effects,ozone layer depletion,photochemical ozone creation,"respiratory effects, inorganics",land use
0,0.004025,2.292553,0.014238,2.356163,0.000141,4.815676e-07,0.001911,0.021735,1.184520e-07,0.057612,8.679305e-07,9.434956e-08,0.006593,0.003075,653.017858


In [21]:
# Here is what the symbolic model looks like 
expr, _ = actToExpression(activity2)
expr

(alpha*(-heat_waste*(alpha - 7) + (alpha + 5*beta)*(alpha*heat_waste + occupation_industrial_area*(alpha + 3))) + 0.4*aluminium_alloy_production_almg3)/alpha

In [23]:
# Fast computation for millions of separate samples
multiLCAWithParamsAlgebric(
    activity2, # The model 
    impacts, # Impacts
    
    # Parameters of the model
    alpha=list(range(1, 1000000)), 
    beta=list(range(1, 1000000)))

Activity 'Occupation, industrial areacopy' was already in 'incer-acv'. Overwriting it
Activity 'Heat, wastecopy' was already in 'incer-acv'. Overwriting it


,"mineral, fossils and renewables",GWP 100a,freshwater and terrestrial acidification,freshwater ecotoxicity,freshwater eutrophication,ionising radiation,marine eutrophication,terrestrial eutrophication,carcinogenic effects,ionising radiation,non-carcinogenic effects,ozone layer depletion,photochemical ozone creation,"respiratory effects, inorganics",land use
0,4.025258e-03,2.292553,1.423843e-02,2.356163,1.408405e-04,4.815676e-07,1.910615e-03,2.173546e-02,1.184520e-07,5.761168e-02,8.679305e-07,9.434956e-08,6.592515e-03,3.075348e-03,3.570179e+02
1,2.012629e-03,1.146276,7.119213e-03,1.178082,7.042023e-05,2.407838e-07,9.553075e-04,1.086773e-02,5.922602e-08,2.880584e-02,4.339653e-07,4.717478e-08,3.296258e-03,1.537674e-03,8.889089e+02
2,1.341753e-03,0.764184,4.746142e-03,0.785388,4.694682e-05,1.605225e-07,6.368717e-04,7.245152e-03,3.948401e-08,1.920389e-02,2.893102e-07,3.144985e-08,2.197505e-03,1.025116e-03,1.599006e+03
3,1.006315e-03,0.573138,3.559607e-03,0.589041,3.521011e-05,1.203919e-07,4.776538e-04,5.433864e-03,2.961301e-08,1.440292e-02,2.169826e-07,2.358739e-08,1.648129e-03,7.688369e-04,2.486854e+03
4,8.050517e-04,0.458511,2.847685e-03,0.471233,2.816809e-05,9.631352e-08,3.821230e-04,4.347091e-03,2.369041e-08,1.152234e-02,1.735861e-07,1.886991e-08,1.318503e-03,6.150695e-04,3.552364e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,4.025279e-09,0.000002,1.423850e-08,0.000002,1.408412e-10,4.815700e-13,1.910625e-09,2.173557e-08,1.184526e-13,5.761197e-08,8.679348e-13,9.435003e-14,6.592548e-09,3.075363e-09,8.879938e+13
999995,4.025275e-09,0.000002,1.423848e-08,0.000002,1.408410e-10,4.815695e-13,1.910623e-09,2.173554e-08,1.184525e-13,5.761191e-08,8.679340e-13,9.434993e-14,6.592542e-09,3.075360e-09,8.879956e+13
999996,4.025271e-09,0.000002,1.423847e-08,0.000002,1.408409e-10,4.815691e-13,1.910621e-09,2.173552e-08,1.184524e-13,5.761186e-08,8.679331e-13,9.434984e-14,6.592535e-09,3.075357e-09,8.879973e+13
999997,4.025266e-09,0.000002,1.423845e-08,0.000002,1.408407e-10,4.815686e-13,1.910619e-09,2.173550e-08,1.184523e-13,5.761180e-08,8.679322e-13,9.434974e-14,6.592529e-09,3.075354e-09,8.879991e+13
